# Environment

Run this cell before any of the cells in the [Experiment](#Environment) section.

In [ ]:
import time
import json
import os
import chi

from datetime import datetime, timedelta
from dateutil import tz

import chi.lease as lease_manager
import chi.server as server_manager
import chi.network as network_manager

# Configure your project site and ID
project_name = 'CH-822154'
site = 'CHI@TACC'

# Configure your testing details
username = '{}TEST'.format(os.getenv('USER'))
image_name = 'CC-Ubuntu16.04'
flavor_name = 'baremetal'
network_name = 'sharednet1'

In [ ]:
class Failure(Exception):
    """Base Class for Test Failure"""

def fail(test_name, message):
    raise Failure('Test {} failed: {}'.format(test_name, message))

# Experiment

Exercises on the various methods in [chi.lease](https://python-chi.readthedocs.io/en/latest/modules/lease.html).

## Functional Interface

### add_device_registration()

### add_fip_reservation()

### add_network_reservation()

### add_node_reservation()

### create_lease()

**CreateLease1**

Create a lease with a single node resevation.
Wait until the lease is active.
Delete the lease.

In [ ]:
chi.set('project_name', project_name)
chi.use_site(site)
test_name = 'CreateLease1'
print()

# Create lease reservations
reservations = []
lease_manager.add_node_reservation(reservations)

# Create lease
lease_name = '{}Lease'.format(username)
try:
    lease_manager.get_lease(lease_name)
    fail(test_name, 'Lease with name {} already exists.'.format(lease_name))
except Failure as f:
    raise f
except:
    pass
lease = lease_manager.create_lease(lease_name, reservations)
print('Created lease {}'.format(lease_name))

# Wait for lease to be active
while lease['status'] == 'PENDING':
    lease = lease_manager.get_lease(lease_name)
    time.sleep(5)
if lease['status'] != 'ACTIVE':
    fail(test_name, 'Lease {} failed to launch. Instead, got lease status {}.'.format(lease_name, lease['status']))
print('Lease {} is active'.format(lease_name))

# Remove lease
lease_manager.delete_lease(lease_name)
try:
    while lease['status'] == 'ACTIVE':
        lease = lease_manager.get_lease(lease_name)
    fail(test_name, 'Tried to delete lease {} and ended up with lease status {}.'.format(lease_name, lease['status']))
except Failure as f:
    raise f
except:
    pass
print('Lease {} successfully removed.'.format(lease_name))

**CreateLease2**

Attempt to create a lease with no reservations. Expect an exception to be raised.

In [ ]:
chi.set('project_name', project_name)
chi.use_site(site)
test_name = 'CreateLease2'
print()

# Create lease
lease_name = '{}Lease'.format(username)
try:
    lease_manager.get_lease(lease_name)
    fail(test_name, 'Lease with name {} already exists.'.format(lease_name))
except Failure as f:
    raise f
except:
    pass
try:
    lease = lease_manager.create_lease(lease_name, [])
    fail(test_name, 'Created lease named {} with no reservations'.format(lease_name))
except Failure as f:
    raise f
except:
    pass
print('Lease with no reservations did not go up')

### delete_lease()

### get_device_reservation()

### get_lease()

### get_lease_id()

### get_node_reservation()

### get_reserved_floating_ips()

### lease_duration()

### wait_for_active()

## Object-oriented Interface

### create_server()

### delete()

### from_existing()

### ready

### refresh()

### status

### wait()